In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np


# Defining our coupled derivatives to integrate

In [ ]:
def dydx(x,y):
    
    #set the derivatives
    
    # our equation is d^2y/dx^2 = -y
    
    # so we can write 
    #dydx = z
    #dzdx = -y
    
    # we will set y  =  y[0]
    # we will sedt z  = y[1]
    
    #declare an array
    y_derivs = np.zeros(2)
    
    #set dydx = z
    y_derivs[0] = y[1]
    
    #set dzdx = -y
    y_derivs[1] = -1*y[0]
    
    #here we have to return an array
    return y_derivs

# Define the Cash-Karpe method

In [ ]:
def cashk_mv_core(dydx,xi,yi,nv,h):
    
    #declare k arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
    
    
  
    
    #declare a temp y array
    y_temp = np.zeros(nv)
    
    
    #get k1 values
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs
    
    #get k2 values
    c2 = 1/5
    a21 = 1/5
    y_temp[:] = yi[:] + a21*k1[:]
    y_derivs = dydx(xi + c2*h,y_temp[:])
    k2[:] = h*y_derivs[:]
    
    
    #get k3 values
    c3 = 3/10
    a31 = 3/40
    a32 = 9/40
    y_temp[:] = yi[:] + a31*k1[:] + a32*k2[:]
    y_derivs = dydx(xi+c3*h,y_temp[:])
    k3[:] = h*y_derivs[:]
    
    
    #get k4 values
    c4 = 3/5
    a41 = 3/10
    a42 = -9/10
    a43 = 6/5
    y_temp[:] = yi[:] + a41*k1[:] + a42*k2[:] + a43*k3[:]
    y_derivs = dydx(xi + c4*h,y_temp)
    k4[:] = h*y_derivs[:]
    
    
    #get k5 values
    c5 = 1
    a51 = -11/54
    a52 = 5/2
    a53 = -70/27
    a54 = 35/27
    y_temp[:] = yi[:] + a51*k1[:] + a52*k2[:] + a53*k3[:] +a54*k4[:]
    y_derivs = dydx(xi + c5*h,y_temp)
    k5[:] = h*y_derivs[:]
    
    
    #get k6 values
    c6 = 7/8
    a61 = 1631/55296
    a62 = 175/512
    a63 = 575/13824
    a64 = 44275/110592
    a65 = 253/4096
    y_temp[:] = yi[:] + a61*k1[:] + a62*k2[:] + a63*k3[:] + a64*k4[:] + a65*k5[:]
    y_derivs = dydx(xi + c6*h,y_temp)
    k6[:] = h*y_derivs[:]
    
    
    
    b1 =37/378
    b2 = 0
    b3 = 250/621
    b4 =125/594
    b5 = 0
    b6 = 512/1771
    yipo =  yi + b1*k1 + b2*k2 + b3*k3 + b4*k4 + b5*k5 + b6*k6
    
    
    
    b1star = 2825/27648
    b2star = 0
    b3star = 18575/48384
    b4star = 13525/55296
    b5star = 277/14336
    b6star = 1/4
    
    yipostar =   yi[:] + b1star*k1[:] + b2star*k2[:] + b3star*k3[:] + b4star*k4[:] + b5star*k5[:] + b6star*k6[:]
    
    Delta = np.fabs(yipo - yipostar)
    

    return yipo,Delta


# Define an adaptive step size driver for Cashk

In [ ]:
def cashk_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    #define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a maximum number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    

    
    #remember the step
    h_step = h
    
    y_2,Delta = cashk_mv_core(dydx,x_i,y_i,nv,h_step)

    #check if Delta is too big
    # and while too big, make smaller
    while(Delta.max()/tol > 1.0):

            
        # will make it smaller
        h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
        y_2,Delta = cashk_mv_core(dydx,x_i,y_i,nv,h_step) # recalcculates Delta, with new h value
            

        if(i>imax):
            print("Too many iterations in cashk_mv_ad()")
            raise StopIteration("Ending after i =",i)
            
            #iterate
        i+=1
        #next time, try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step we actually took
    return y_2,h_new,h_step
        

# Define a wrapper for cashk

In [ ]:
def cashk_mv(dfdx,a,b,y_a,tol):
    
    #dfdx is the derivative wrt x
    # a is the lower bound
    #b is the upper bound
    #y_a are the boundary conditions
    #tol is the tolerance for integrating y
    
    #define our starting step
    xi = a
    yi = y_a.copy()
    
    #an initial step size == make very small
    h = 1.0e-4 * (b-a)
    
    #set a maxmimum number of iterations
    imax = 10000
    
    # set an iteration variable
    i = 0
    #set the number of coupled odes to the 
    #size of y_a
    nv = len(y_a)
    
    #set the initial conditions
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    #Set a flag
    flag =1 
    
    #loop until we reach the right side
    while(flag):
        
        #calculate y_i+1
        yi_new, h_new,h_step = cashk_mv_ad(dydx,xi,yi,nv,h,tol)
        
        #update the step
        h = h_new
        
        #prevent an overshoot
        if(xi+h_step>b):
            
            #take a smaller step
            h = b-xi
            
            #recalculate y_i+1
            yi_new, h_new, h_step = cashk_mv_ad(dydx,xi,yi,nv,h,tol)
            
            #break
            flag = 0
        
        
        #update values
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y
        y = y_new
        
        # prevent too many iterations
        if(i>=imax):
            
            print("Maximum iterations reached")
            raise StopIteration("Iteration number =",i)
        
        #iterate
        i += 1
        #output some information
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i,xi,h_step,b)
        print(s)
        
        #break if new xi is == b
        if(xi==b):
            flag = 0
    
    #return answer
    return x,y

In [ ]:
a = 0.0
b = 2.0 * np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

#perform the integration
x,y = cashk_mv(dydx,a,b,y_0,tolerance)

# Plot the result

In [ ]:
plt.plot(x,y[:,0],'o',label='y(x)')
plt.plot(x,y[:,1],'o', label='dydx(x)')
xx = np.linspace(0,2.0*np.pi,1000)
plt.plot(xx,np.sin(xx),label='sin(x)')
plt.plot(xx,np.cos(xx),label='cos(x)')
plt.xlabel('x')
plt.ylabel('y,dydx')
plt.legend(frameon=False)
plt.plot()

# Plot in error
## Errors will exceed tolerance

In [ ]:
sine = np.sin(x)
cosine = np.cos(x)
y_error = (y[:,0]-sine)
dydx_error = (y[:,1]-cosine)
plt.plot(x,y_error, label="y(x) Error")
plt.plot(x,dydx_error, label = "dydx(x) Error")
plt.legend(frameon=False)
plt.show()